In [28]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import tensorflow as tf
import os
from collections import defaultdict

In [29]:
def augment_image(image):
    augmented_images = []
    augmented_images.append(image)
    augmented_images.append(tf.image.flip_left_right(image))
    augmented_images.append(tf.image.rot90(image))
    augmented_images.append(tf.image.flip_up_down(image))
    
    return augmented_images

In [30]:
def preprocess_and_extract_features(images, labels):
    sift = cv2.SIFT_create()
    descriptors = []
    descriptor_labels = []
    
    for img, label in zip(images, labels):
        try:
            if isinstance(img, tf.Tensor):
                img = img.numpy()
            resized_img = cv2.resize(img, (224, 224))
            
            if np.max(resized_img) <= 1.0:
                resized_img = (resized_img * 255).astype(np.uint8)
            else:
                resized_img = resized_img.astype(np.uint8)
            keypoints, desc = sift.detectAndCompute(resized_img, None)
            
            if desc is not None and len(desc) > 0:
                descriptors.append(desc)
                descriptor_labels.append(label)
            else:
                print(f"No descriptors found for label {label}. Skipping this image.")

        except Exception as e:
            print(f"Error processing image with label {label}: {e}")
    
    if len(descriptors) == 0:
        raise ValueError("No valid descriptors found in the dataset.")
    
    combined_descriptors = np.vstack(descriptors)
    combined_labels = np.hstack([[label] * len(desc) for label, desc in zip(descriptor_labels, descriptors)])
    
    return combined_descriptors, combined_labels

In [31]:
# Load dataset
dataset_path = "./batik_assets/"
images = []
class_list = []
label_class = ['batik-bali', 'batik-betawi', 'batik-celup', 'batik-cendrawasih', 'batik-ceplok', 'batik-ciamis', 'batik-garutan', 'batik-gentongan', 'batik-kawung', 'batik-keraton', 'batik-lasem', 'batik-megamendung', 'batik-parang', 'batik-pekalongan', 'batik-priangan', 'batik-sekar', 'batik-sidoluhur', 'batik-sidomukti', 'batik-sogan', 'batik-tambal']
class_images = defaultdict(list)

In [32]:
for i, class_name in enumerate(label_class):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for file in os.listdir(class_path):
            try:
                file_path = os.path.join(class_path, file)
                image = load_img(file_path)
                image = img_to_array(image)
                image = tf.image.resize(image, (256, 256)) / 255.0
                class_images[class_name].append((image, i))

            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

In [33]:
all_images = []
all_labels = []

for class_name, images_and_labels in class_images.items():
    for img, label in images_and_labels:
        all_images.append(img.numpy())
        all_labels.append(label)

all_images = np.array(all_images)
all_labels = np.array(all_labels)

train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, stratify=all_labels,random_state=42
)

In [34]:
augmented_train_images = []
augmented_train_labels = []

for img, label in zip(train_images, train_labels):
    augmented_images = augment_image(tf.convert_to_tensor(img))
    for aug_image in augmented_images:
        augmented_train_images.append(aug_image.numpy())
        augmented_train_labels.append(label)

augmented_train_images = np.array(augmented_train_images)
augmented_train_labels = np.array(augmented_train_labels)

descriptors_train, labels_train = preprocess_and_extract_features(augmented_train_images, augmented_train_labels)
descriptors_test, labels_test = preprocess_and_extract_features(test_images, test_labels)

In [35]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(descriptors_train, labels_train)

y_pred_knn = knn_model.predict(descriptors_test)
print("KNN Classification Report:")
print(classification_report(labels_test, y_pred_knn))
print("KNN Accuracy:", accuracy_score(labels_test, y_pred_knn))

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.08      0.09      8570
           1       0.16      0.13      0.14      8507
           2       0.06      0.05      0.05      4366
           3       0.06      0.05      0.06      4382
           4       0.15      0.16      0.16      5913
           5       0.25      0.32      0.28      6997
           6       0.10      0.08      0.09      7105
           7       0.06      0.05      0.06      7189
           8       0.18      0.17      0.18      5405
           9       0.07      0.08      0.08      8796
          10       0.14      0.13      0.13     10528
          11       0.47      0.45      0.46      6742
          12       0.25      0.40      0.31      6605
          13       0.13      0.10      0.11      8441
          14       0.03      0.03      0.03      7001
          15       0.08      0.07      0.07      3489
          16       0.17      0.34      0.23      5031


In [36]:
import pickle

knnPickle = open('knn_model.pkl', 'wb')
pickle.dump(knn_model, knnPickle)
knnPickle.close()